In [1]:
import pandas as pd
import json
import math

In [ ]:
#This script is to extract batch results and compare to the manually classified set
#The comparison allows to determine the accuracy of the model in correctly classifying the tweets 

In [15]:
file_path = 'D:\Thesis\\batch_output\\'
file_name = '4o-historical.jsonl'

#an empty list to store results from the json batches
data = []

with open(file_path + file_name, 'r') as file:
    for line in file:
        record = json.loads(line)
        if 'custom_id' in record and 'response' in record:
            response = record['response']
            if 'body' in response and 'choices' in response['body']:
                choices = response['body']['choices']
                if len(choices) > 0 and 'message' in choices[0]:
                    message = choices[0]['message']
                    if 'content' in message:
                        data.append({'line_id': record['custom_id'], 'content': message['content']})

df = pd.DataFrame(data)

In [ ]:
df['line_id'] = df['line_id'].str.replace('request-', '')
df.head()

In [18]:
df.to_csv('4o-historical.csv', index=False)

In [ ]:
#read in both csv files, the batch output and the manual classification
df1 = pd.read_csv('4o-historical.csv')
df2 = pd.read_csv('bitcoin_1000_random1.csv')

df1.head()

In [ ]:
#append content column from df1 to df2
df2['content'] = df1['content']
df2.head()

In [ ]:
#only take the first 1000 records
df2 = df2[:1000]
df2.head(10)


In [ ]:
#convert sentiment column to int
df2['sentiment'] = df2['sentiment'].astype(float)
df2.head()

In [ ]:
#Change the polarity score to a sentiment score (1, 0, -1) for the accuracy check
df2['content'] = df2['content'].astype(float)
def floor_ceil(value):
    if value < 0.01:
        return math.floor(value)
    elif value > 0.01:
        return math.ceil(value)
    else:
        return 0.0

df2['content'] = df2['content'].apply(floor_ceil)

df2.head()

In [7]:
#convert 
df2['content'] = df2['content'].astype(float)
df2.head()

,id,text,cleanText,sentiment,content
0,1,Blue Ridge Bank shares halted by NYSE after #b...,blue ridge bank shares halted nyse bitcoin atm...,1.0,0.0
1,2,"😎 Today, that's this #Thursday, we will do a ""...",today thursday take friend leowandersleb btc w...,0.0,0.0
2,3,"Guys evening, I have read this article about B...",guys evening read article btc would like share,0.0,0.0
3,4,$BTC A big chance in a billion! Price: \487264...,btc big chance billion price bitcoin btc crypto,1.0,1.0
4,5,This network is secured by 9 508 nodes as of t...,network secured nodes today soon biggest bears...,1.0,1.0


In [ ]:
#check the accuracy of the model
sum = 0
total = len(df2)
for __, row in df2.iterrows():
    if row['sentiment'] == row['content']:
        sum += 1
print(sum)
print(f'The accuracy is: {(sum/total)*100}%')

In [9]:
#check the models classification
df2.groupby('content').count()

,id,text,cleanText,sentiment
content,,,,
-1.0,102,102,102,102
0.0,421,421,421,421
1.0,227,227,227,227


In [11]:
#check the manual classification
df2.groupby('sentiment').count()

,id,text,cleanText,content
sentiment,,,,
-1.0,100,100,100,100
0.0,425,425,425,425
1.0,225,225,225,225
